## Autogluon fine tuning 예시

- 새로운 class에 대해서 성능이 잘 나오는 object detection model을 fine-tuning 하는 예시입니다 : [참고](https://auto.gluon.ai/stable/tutorials/multimodal/object_detection/finetune/detection_high_performance_finetune_coco.html)
- 앞의 노트북에서 autogluon을 설치했다고 가정합니다. (설치하지 않았다면 앞의 노트북 참고해서 설치할 것)
- 앞의 노트북에서 데이터를 다운로드 했다고 가정합니다.



In [ ]:
!pip list | grep autogluon

In [ ]:
import os
import time

In [ ]:
data_dir = os.path.join("data", "pothole")
train_path = os.path.join(data_dir, "Annotations", "usersplit_train_cocoformat.json")
val_path = os.path.join(data_dir, "Annotations", "usersplit_val_cocoformat.json")
test_path = os.path.join(data_dir, "Annotations", "usersplit_test_cocoformat.json")

In [ ]:
from mmdet.apis import init_detector, inference_detector
import mmcv

In [ ]:
from autogluon.multimodal import MultiModalPredictor

In [ ]:
# checkpoint_name = "vfnet_r50_fpn_mdconv_c3-c5_mstrain_2x_coco"  # 튜토리얼 사용 모델
# checkpoint_name = "vfnet_r50_fpn_mstrain_2x_coco"
checkpoint_name = "yolox_x_8x8_300e_coco"
num_gpus = -1  # use all GPUs
val_metric = "map"

In [ ]:
predictor = MultiModalPredictor(
    hyperparameters={
        "model.mmdet_image.checkpoint_name": checkpoint_name,
        "env.num_gpus": num_gpus,
        "optimization.val_metric": val_metric,
    },
    problem_type="object_detection",
    sample_data_path=train_path,
    path="./ag-trained-models"
)

In [ ]:
import time
start = time.time()
predictor.fit(
    train_path,
    hyperparameters={
        "optimization.learning_rate": 5e-6, # we use two stage and detection head has 100x lr
        "optimization.max_epochs": 1,   # for the real use case, at least 50
        "optimization.check_val_every_n_epoch": 1, # make sure there is at least one validation
        "env.per_gpu_batch_size": 2,  # decrease it when model is large
    },
)
end = time.time()

In [ ]:
print("This finetuning takes %.2f seconds." % (end - start))

In [ ]:
# Load Trained Predictor from S3
# zip_file = "https://automl-mm-bench.s3.amazonaws.com/object_detection/checkpoints/pothole_AP50_718.zip"
# download_dir = "./pothole_AP50_718"
# load_zip.unzip(zip_file, unzip_dir=download_dir)
# better_predictor = MultiModalPredictor.load("./pothole_AP50_718/AutogluonModels/ag-20221123_021130") # 저자들이 미리 50 epoch으로 학습한 모델이다.
better_predictor = MultiModalPredictor.load("./AutogluonModels/ag-20230327_141514/")   # yolox 150 epoch
better_predictor.set_num_gpus(1)

# Evaluate new predictor
better_predictor.evaluate(test_path)

In [ ]:
pred = better_predictor.predict(test_path)

In [ ]:
pred

In [ ]:
from autogluon.multimodal.utils import visualize_detection
from PIL import Image
from IPython.display import display

conf_threshold = 0.25  # Specify a confidence threshold to filter out unwanted boxes
visualization_result_dir = "./pothole-test-sample"  # Use the pwd as result dir to save the visualized image

In [ ]:

visualized = visualize_detection(
    pred=pred.iloc[:30],
    detection_classes=predictor.get_predictor_classes(),
    conf_threshold=conf_threshold,
    visualization_result_dir=visualization_result_dir,
)


In [ ]:
for i in range(30):
    img = Image.fromarray(visualized[i][:, :, ::-1], 'RGB')
    display(img)